In [1]:
# Standard Imports
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import pickle
import eli5

# Transformers
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

# Modeling Evaluation
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, confusion_matrix, classification_report
from IPython.display import display, Markdown

# Pipelines
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from imblearn.pipeline import Pipeline as PipeImb
from imblearn.pipeline import make_pipeline


# Machine Learning
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier

#Validation methods
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedStratifiedKFold

#Sampling methods
from imblearn.over_sampling import SMOTENC

#Feature selection
from sklearn.feature_selection import RFE



In [2]:
#Look at pipeline and what it does
# from sklearn import set_config
#set_config(display='diagram')
#display(search2)

# User defined functions

In [3]:
def a_xy_split(dataframe,y_name,test_size=.3):
    '''Conduct XY and Train/Test splits. Also generates helper variables that are needed by other functions,
    such as categorical and numerical column name lists, and a boolean list of categorical columns for SMOTE transformers


    Parameters
    ---------------
    dataframe (dataframe): Cleaned dataframe ready for modelling
    test_size (float): Number as percent (.n) of sample to keep for test set
    
    Returns
    ---------------
    X_train, X_test, y_train, y_test,numerical_columns,categorical_columns,cat_col_bool

    '''
    
    #Get xy split and train/test data ready
    X = dataframe.drop([y_name], axis=1)
    y = dataframe.attrition
    y = LabelEncoder().fit_transform(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,random_state=42)

    #Prep numerical and other column types for use in preprocessor
    numerical=X.select_dtypes(include='number')
    categorical=X.select_dtypes(include='object')
    numerical_columns=list(numerical.columns)
    categorical_columns=list(categorical.columns)
    columns=list(X.columns)
    cat_col_bool=[True if x in categorical_columns else False for x in columns]

    return X_train, X_test, y_train, y_test,numerical_columns,categorical_columns,cat_col_bool

In [4]:
def b_preprocessors():
    '''Build column transformer for numerical (StandardScaler) and categorical (OHE) data and return it as a processor

    Parameters
    ---------------
    none: Requires creation of numerical_columns and categorical_columns variables in the xysplit
    
    Returns
    ---------------
    preprocessor object

    '''
    #Creating ss transformer to scale the continuous numerical data with StandardScaler()
    ss = Pipeline(steps=[('ss', StandardScaler())])

    #Creating ohe transformer to encode the categorical data with OneHotEncoder()
    ohe = Pipeline(steps=[('ohe', OneHotEncoder(handle_unknown='ignore', drop='first'))])

    #Creating preprocess column transformer to combine the ss and ohe pipelines
    preprocess = ColumnTransformer(
                        transformers=[
                            ('num', ss, numerical_columns),
                            ('cat', ohe, categorical_columns)
                        ])
    return preprocess



In [5]:
def c_get_standard_params():
    '''Get typical parameters for tuning hyperparameters for most classification types.
    Used in next steps when conducting grid feature.
    
    Parameters
    ---------------
    none

    Returns
    ---------------
    rand_forest_parms,xgb_params,log_regress_params,knn_params,gaussian_params,bagging_params,decision_tree_params
    '''

    #Creating parameter grid for Random Forest
    rand_forest_parms = {'classifier__n_estimators': [100, 300, 500],
    'classifier__max_depth':[6, 25, 50, 70],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 10]}

    decision_tree_params = {'classifier__max_depth':[6, 25, 50, 70],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 10],
    'classifier__max_features': [1,2,5,10]}

    xgb_params = { 'classifier__max_depth': [3,6,10],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'classifier__n_estimators': [100, 500, 1000],
    'classifier__colsample_bytree': [0.3, 0.7],
    'classifier__subsample':[.6,1]}

    log_regress_params={'classifier__solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
    'classifier__penalty':['none','l1','l2','elasticnet'],
    'classifier__C':[100,10,1.0, 0.1,0.01]}

    knn_params={'classifier__n_neighbors':[1, 5, 10],
    'classifier__weights': ['uniform', 'distance'],
    'classifier__metric': ['euclidean', 'manhattan', 'minkowski']}

    gaussian_params={'classifier__var_smoothing': [1e-11, 1e-10, 1e-9]}

    bagging_params= {'classifier__base_estimator__max_depth' : [1, 2, 3, 4, 5],
    'classifier__max_samples' : [0.05, 0.1, 0.2, 0.5]}

    return rand_forest_parms,xgb_params,log_regress_params,knn_params,gaussian_params,bagging_params,decision_tree_params

In [6]:
def _evaluation(y, y_hat, title = 'Confusion Matrix',silent=False):
    '''Creates table and graph for classification outcomes.  Generates a confusion matrix
    
    Parameters
    ---------------
    y (array): y value, either train, validation, or test
    y_hat (array): y predicted value, either train, validation, or test

    Returns
    ---------------
    Nothing.  Prints results and confusion matrix
    '''

    cm = confusion_matrix(y, y_hat)
    precision = precision_score(y, y_hat)
    recall = recall_score(y, y_hat)
    accuracy = accuracy_score(y,y_hat)
    f1 = f1_score(y,y_hat)
    if silent==False:
        print('Accuracy: ', accuracy)
        print('Recall: ', recall)
        print('Precision: ', precision)
        print('F1: ', f1)

    sns.heatmap(cm,  cmap= 'PuBu', annot=True, fmt='g', annot_kws=    {'size':20})
    plt.xlabel('predicted', fontsize=18)
    plt.ylabel('actual', fontsize=18)
    plt.title(title, fontsize=18)
    
    plt.show()



# Stack formula

In [7]:
def db_check_best_model(models='default',smote_type="nc"):
    '''Compares default models to determine which have the strongest predictive power.
    Prints out results per model, and a summary table sorted by F1 score.
    
    Parameters
    ---------------
    models (list): List of models to include.  If default, then a standard list is used instead.
    smote_type (string): Type of SMOTE to include if upsampling is necessary

    Returns
    ---------------
    Dataframe of results sorted by F1 values.
    '''
    
    if models=='default':
        models=[LogisticRegression(),RandomForestClassifier(),XGBClassifier(),KNeighborsClassifier(),DecisionTreeClassifier(),SVC(),GaussianNB(),BaggingClassifier()]
    avg_acc=[]
    avg_rec=[]
    avg_prec=[]
    f1=[]
    for i in models:
        print (i)
        a,r,p,f=e_cross_validate_grid(preprocess,i,cv=KFold(),smote_type=smote_type,check_best=True)
        avg_acc.append(a)
        avg_rec.append(r)
        avg_prec.append(p)
        f1.append(f)
        print()
    res_df=pd.DataFrame(columns=["model",'accuracy','recall','precision','f1'])
    res_df['model']=models
    res_df['accuracy']=avg_acc
    res_df['recall']=avg_rec
    res_df['precision']=avg_prec
    res_df['f1']=f1
    return res_df.sort_values(by="f1",ascending=False)

In [8]:
def e_cross_validate_grid(preprocessor, classifier, smote_type='NC', cv=KFold(), param_grid="none", scoring="recall", conf_matrix=False, check_best=False):
    '''Core feature of Classification Modelling.  Conducts train and cross-validation analyses using pipeline provided and outputs averaged results.

    - Use 'param_grid' with 'scoring' attribute to tune hyperparameters with GridSearchCV
    - Leave param_grid as 'none' to instead produce a cross-validated result for one classifier. Typically if already tuned. Output can be used in the test function
    
    Parameters
    ---------------
    preprocessor (ColumnTransformer): Specific ColumnTransformer to apply to scale and code data
    classifier (Classification Object): The specific classifier to use (e.g. LogisticRegression() )
    smote_type (string): SMOTE to use before preprocessor is included.  Upsamples imbalanced data. NC for numerical-categorical by default.
    cv (cross-validation function): Default is KFold()
    
    param_grid (dictionary): Dictionary of parameters to include i GridSearchCV.  Can get default dictionaries from get_standard_params function.
    scoring (string): type of scoring to optimize as part of GridSearchCV
    
    conf_matrix (boolean): Whether to output a confusion matrix.  Default false
    check_best (boolean): Used by another function. Outputs accuracy,recall,precision,F1 as variables instead of outputting a pipeline. False by default
    
    Returns
    ---------------
    Pipeline fitted on data, to be used in predicting test data
    If check_best=True, instead outputs accuracy, recall, precision, and F1 scores

    '''
    
    cv = cv

    #Set correct pipeline for use in the formula
    pipeline = PipeImb(steps=[
    ('preprocess', preprocessor),
    ('classifier', classifier)])


    if smote_type.lower()=="nc":
        pipeline.steps.insert(0,('smote',SMOTENC(categorical_features=X_train.dtypes==object,random_state=1234,n_jobs=-1)))


    if param_grid!='none':
        search = GridSearchCV(pipeline, param_grid=param_grid, cv=cv,scoring=scoring)
    else:
        search=pipeline
# Run train and validation methods, print average result for each
    train_acc = []
    train_rec=[]
    train_prec=[]
    test_acc = []
    test_rec=[]
    test_prec=[]
    for train_ind, val_ind in cv.split(X_train, y_train):
        X_t, y_t = X_train.iloc[train_ind], y_train[train_ind]
        search.fit(X_t, y_t)
        y_hat_t = search.predict(X_t)
        train_acc.append(accuracy_score(y_t, y_hat_t))
        train_rec.append(recall_score(y_t,y_hat_t))
        train_prec.append(precision_score(y_t,y_hat_t))
        X_val, y_val = X_train.iloc[val_ind], y_train[val_ind]
        y_hat_val = search.predict(X_val)
        test_acc.append(accuracy_score(y_val, y_hat_val))
        test_rec.append(recall_score(y_val, y_hat_val))
        test_prec.append(precision_score(y_val, y_hat_val))

    if conf_matrix==True:
        print("Validation set confusion matrix")
        print(_evaluation(y_val, y_hat_val,silent=True))
        print()
    print('Avg Training Accuracy: {}'.format(np.mean(train_acc)))
    print("Avg Training Recall: {}".format(np.mean(train_rec)))
    print("Avg Training Precision: {}".format(np.mean(train_prec)))
    print()
    print('Avg Validation Accuracy: {}'.format(np.mean(test_acc)))
    print("Avg Validation Recall: {}".format(np.mean(test_rec)))
    print("Avg Validation Precision: {}".format(np.mean(test_prec)))
    print()

# If grid search used, print best estimator's features
    if (param_grid!='none'):
        ohe_cols = list(search.best_estimator_.named_steps['preprocess'].named_transformers_['cat'].named_steps['ohe'].get_feature_names_out(
        input_features=categorical_columns))
        num_feats = list(numerical_columns)
        num_feats.extend(ohe_cols)
        feat_imp = eli5.explain_weights_df(search.best_estimator_.named_steps['classifier'], top=10, feature_names=num_feats)
        print(feat_imp)
        print()
        print('Grid Search Best Params:')
        print()
        print(search.best_params_)
    if check_best==True:
        acc= np.mean(test_acc)
        rec=np.mean(test_rec)
        prec=np.mean(test_prec)
        f1=2*(prec*rec)/(prec+rec)

        return acc,rec,prec,f1
    else:
        return search


In [9]:
def g_validate_using_test(preprocessor, classifier, param_grid, cv=KFold(),smote_type='nc',scoring='roc_auc'):
    '''Creates a pipeline using GridSearchCV, then applies the result immediately to the test data. 
    Prints results for both
    
    Parameters
    ---------------
    preprocessor (ColumnTransformer): Specific ColumnTransformer to apply to scale and code data
    classifier (Classification Object): The specific classifier to use (e.g. LogisticRegression() )
    param_grid (dictionary): Dictionary of parameters to include i GridSearchCV.  Can get default dictionaries from get_standard_params function.
    cv (cross-validation function): Default is KFold()
    smote_type (string): SMOTE to use before preprocessor is included.  Upsamples imbalanced data. NC for numerical-categorical by default.
    scoring (string): type of scoring to optimize as part of GridSearchCV    

    Returns
    ---------------
    Results of test for train/test

    '''
    
    final_pipeline = GridSearchCV(PipeImb(steps=[
            ('preprocess', preprocessor),
            ('classifier', classifier)
        ]), param_grid, cv=cv,scoring=scoring)
    
    if smote_type.lower()=='nc':
        final_pipeline.steps.insert(0,('smote',SMOTENC(categorical_features=X_train.dtypes==object,random_state=1234,n_jobs=-1)))

    
    #Fit and predict on train data
    final_pipeline.fit(X_train, y_train)

    train_pred = final_pipeline.best_estimator_.predict(X_train)
    cv_score=final_pipeline.best_score_
    #train_pred=final_pipeline.predict(X_train)

    print('Evaluation on training data \n')
    print(_evaluation(y_train, train_pred))
    print('Cross-validation score train',cv_score)
    print('\n')
    #Predict on test data

    test_pred = final_pipeline.best_estimator_.predict(X_test)
    test_score=final_pipeline.score(X_test,y_test)
    #test_pred = final_pipeline.predict(X_test)
    print('Evaluation on testing data \n')
    print(_evaluation(y_test, test_pred))
    print("Test score", test_score)

In [10]:
def f_validate_pipeline_on_test(pipeline):
    '''Creates a pipeline using GridSearchCV, then applies the result immediately to the test data. 
    Prints results for both
    
    Parameters
    ---------------
    pipeline (fitted pipeline): Pipeline, which must have been fitted on the X_train data already

    Returns
    ---------------
    Results of test for train/test evaluation for that pipeline
    '''

    test_pred = pipeline.predict(X_test)
    #test_pred = final_pipeline.predict(X_test)
    print('Evaluation on testing data \n')
    print(_evaluation(y_test, test_pred))

In [11]:
def da_RFE_find_features(min_r=1,max_r='all',smote_type='nc',scoring='recall'):
    '''Finds best number of features to keep in the dataset.  May or may not be helpful information.
    
    Parameters
    ---------------
    min and max features to keep.  Default range is 1-max features
    smote type, default nc
    scoring type for the cross validation process to return

    Returns
    ---------------
    Results of test for train/test evaluation for that pipeline
    '''
    results_df=pd.DataFrame(columns=['num_n',scoring])

    if smote_type.lower()=="nc":
        smote=SMOTENC(categorical_features=X_train.dtypes==object,random_state=1234,n_jobs=-1)
        X_res,y_res=smote.fit_resample(X_train,y_train)
    
        X_train_scaled=preprocess.fit_transform(X_res,y_res)
        X_test_scaled=preprocess.transform(X_test)
    else:
        X_train_scaled=preprocess.fit_transform(X_train,y_train)
        X_test_scaled=preprocess.transform(X_test)
        y_res=y_train
    
    feature_names=preprocess.get_feature_names_out()
    X_train_scaled=pd.DataFrame(data=X_train_scaled,columns=feature_names)
    X_test_scaled=pd.DataFrame(data=X_test_scaled,columns=feature_names)
    if max_r=='all':
        max_r=len(X_train_scaled.columns)

    for i in range(min_r,max_r):
        model=RFE(DecisionTreeClassifier(random_state=1234),n_features_to_select=i,step=1)
        model.fit(X_train_scaled,y_res)

        kept_features = model.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
        rfe_kept_features = list(X_train_scaled.iloc[:,kept_features].columns)
        X_temp=X_train_scaled[rfe_kept_features].copy()
        X_test_temp=X_test_scaled[rfe_kept_features].copy()
        mod_used=DecisionTreeClassifier(random_state=1234)
        train_score=cross_val_score(mod_used, X_temp, y_res, cv=5,scoring=scoring)
        test_score=cross_val_score(mod_used, X_test_temp, y_test, cv=5,scoring=scoring)

        mean_r2=((train_score.mean()+test_score.mean())/2)
        mean_r2=round(mean_r2,2)
        results_df.loc[i]=i,mean_r2
    return results_df.sort_values(by=scoring,ascending=False),rfe_kept_features

# Create stacker

In [12]:
def h_build_stacking_classifier(model_list):
    '''Function to build a stacking classifier.  Format is as follows:
        
    [('dt',DecisionTreeClassifier()), ('log',LogisticRegression())]

    Parameters
    ---------------
    model list (list): In format above

    Returns
    ---------------
    Classifier object
    '''
    
    # define the base models
    level0 = list()
    for i in model_list:
        level0.append(i)
    
    # define meta learner model
    level1 = LogisticRegression(random_state=1234)
    
    # define the stacking ensemble
    model = StackingClassifier(estimators=level0, final_estimator=level1, cv=5,n_jobs=-1)
    return model

# Create VotingClassifier

In [13]:
def i_build_voting_classifier(model_list, weights=None):
    '''Function to build a voting classifier with weights.  Format is as follows:
        
    clf1=LogisticRegression(random_state=1234)
    clf2=SVC(probability=True,random_state=1324)
    model_list=[('log',clf8),('SVC',clf3)]
    weights=[100,75,100]

    Parameters
    ---------------
    model list (list): In format above
    weights (list): in format above

    Returns
    ---------------
    Soft Voting and Hard Voting classifier objects
    '''
    
    clf_soft=VotingClassifier(estimators=model_list,weights=weights,voting='soft',n_jobs=-1)
    clf_hard=VotingClassifier(estimators=model_list,weights=weights,voting='hard',n_jobs=-1)
    return clf_soft,clf_hard



# Code here

In [14]:
df=pd.read_csv(r"Data\4b.IBM_data_cleaned_dropped - Copy - Copy.csv")
df2=df[df['age']<42].copy()

In [24]:
y_name='attrition'
X_train, X_test, y_train, y_test,numerical_columns,categorical_columns,cat_col_bool=a_xy_split(df2,y_name,test_size=.3)
preprocess=b_preprocessors()
rand_forest_parms,xgb_params,log_regress_params,knn_params,gaussian_params,bagging_params,decision_tree_params=c_get_standard_params()

In [16]:
test,keep=da_RFE_find_features(scoring='roc_auc')
test2,keep=da_RFE_find_features(scoring='recall')

In [17]:
test

,num_n,roc_auc
30,30.0,0.73
13,13.0,0.73
12,12.0,0.73
45,45.0,0.72
11,11.0,0.72
27,27.0,0.72
32,32.0,0.72
20,20.0,0.72
18,18.0,0.72
29,29.0,0.72


In [172]:
test2

,num_n,recall
28,28.0,0.61
30,30.0,0.61
27,27.0,0.61
13,13.0,0.61
12,12.0,0.61
16,16.0,0.60
11,11.0,0.60
18,18.0,0.60
29,29.0,0.59
32,32.0,0.59


In [18]:
test,keep=da_RFE_find_features(min_r=13,max_r=14,scoring='recall')

In [174]:
test

,num_n,recall
13,13.0,0.61


In [175]:
keep

['num__age',
 'num__distancefromhome',
 'num__education',
 'num__environmentsatisfaction',
 'num__jobinvolvement',
 'num__jobsatisfaction',
 'num__monthlyincome',
 'num__numcompaniesworked',
 'num__relationshipsatisfaction',
 'num__stockoptionlevel',
 'num__yearswithcurrmanager',
 'cat__maritalstatus_single',
 'cat__overtime_yes']

In [22]:
keep=['age',
 'distancefromhome',
 'education',
 'environmentsatisfaction',
 'jobinvolvement',
 'jobsatisfaction',
 'monthlyincome',
 'numcompaniesworked',
 'relationshipsatisfaction',
 'stockoptionlevel',
 'yearswithcurrmanager',
 'maritalstatus',
 'overtime','attrition']

In [25]:
v1=db_check_best_model()

df_rfe=df2[keep].copy()

LogisticRegression()
Avg Training Accuracy: 0.8652505561409672
Avg Training Recall: 0.7093397085124342
Avg Training Precision: 0.6379918892753117

Avg Validation Accuracy: 0.8303513186096356
Avg Validation Recall: 0.6435675618411991
Avg Validation Precision: 0.5520904335420465


RandomForestClassifier()
Avg Training Accuracy: 1.0
Avg Training Recall: 1.0
Avg Training Precision: 1.0

Avg Validation Accuracy: 0.8426800857329232
Avg Validation Recall: 0.46539816727580907
Avg Validation Precision: 0.6365965556182948


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
   

In [34]:
X_train, X_test, y_train, y_test,numerical_columns,categorical_columns,cat_col_bool=a_xy_split(df_rfe,y_name,test_size=.3)
preprocess=b_preprocessors()


In [35]:
v2=db_check_best_model()

LogisticRegression()
Avg Training Accuracy: 0.8067732115677322
Avg Training Recall: 0.6507324406412128
Avg Training Precision: 0.501851869747454

Avg Validation Accuracy: 0.7920417482061317
Avg Validation Recall: 0.6150181330225823
Avg Validation Precision: 0.47023970609336463


RandomForestClassifier()
Avg Training Accuracy: 1.0
Avg Training Recall: 1.0
Avg Training Precision: 1.0

Avg Validation Accuracy: 0.8248998229428757
Avg Validation Recall: 0.4690899588608154
Avg Validation Precision: 0.5696576081707045


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=None,
              gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
    

In [36]:
v1

,model,accuracy,recall,precision,f1
0,LogisticRegression(),0.830351,0.643568,0.552090,0.594330
5,SVC(),0.835821,0.533555,0.592015,0.561267
2,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.830370,0.503350,0.585892,0.541493
1,"(DecisionTreeClassifier(max_features='auto', r...",0.842680,0.465398,0.636597,0.537699
3,KNeighborsClassifier(),0.779722,0.587133,0.449896,0.509434
7,(DecisionTreeClassifier(random_state=142459846...,0.802991,0.386772,0.497578,0.435233
4,DecisionTreeClassifier(),0.740052,0.495243,0.363577,0.419317
6,GaussianNB(),0.395285,0.893300,0.229375,0.365023


In [37]:
v2

,model,accuracy,recall,precision,f1
0,LogisticRegression(),0.792042,0.615018,0.470240,0.532972
5,SVC(),0.809841,0.531547,0.514264,0.522763
1,"(DecisionTreeClassifier(max_features='auto', r...",0.824900,0.469090,0.569658,0.514505
2,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.816653,0.480262,0.548786,0.512243
6,GaussianNB(),0.756500,0.612441,0.416334,0.495696
3,KNeighborsClassifier(),0.763340,0.596649,0.421399,0.493939
7,(DecisionTreeClassifier(random_state=139238130...,0.809841,0.443796,0.536849,0.485907
4,DecisionTreeClassifier(),0.777020,0.504295,0.434107,0.466576


In [39]:
cv=RepeatedStratifiedKFold(random_state=1234)
e_cross_validate_grid(preprocess,LogisticRegression(),param_grid=log_regress_params,scoring='roc_auc',cv=cv)

c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
c:\Users\MichaelTaylo_c9zoof3\anaconda3\envs\DA_Env\lib\site-packages\sklearn\linear_model\_logistic.py:1483: UserWarning: Setting penalty='none' will ignore the C and l1_ratio par